## Grad-CAM Visual Explanations

Up to this point we have trained and evaluated our models (MobileNetV2 and DenseNet121) using standard metrics such as accuracy, precision, recall and AUC.  
While these numbers are useful, they don’t tell us **where** the model is looking in the image when making its decision.  

To add explainability, we use **Grad-CAM (Gradient-weighted Class Activation Mapping)**.  
This method produces a heatmap that highlights the areas of a mammogram image that most influenced the model’s prediction.  

The outputs we generate here will help us:
- Confirm if the model is focusing on the correct regions (e.g. lesion areas).
- Inspect both correct and incorrect predictions (True Positives, True Negatives, False Positives, False Negatives).
- Provide clear visuals for the final report and evaluation chapter.

The following cells implement Grad-CAM and display three panels for each selected case:
1. The original input image.  
2. The heatmap of important regions.  
3. An overlay of the heatmap on top of the original image.  

These visualisations give us a more intuitive understanding of the model’s behaviour, beyond raw metrics.


In [23]:
# --- Core ---
import os, glob, random, json
from pathlib import Path
import re

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# --- ML / DL ---
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc

# --- Keras (we’ll import DenseNet later) ---
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import Precision, Recall

# Reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# --- Project paths ---

ROOT = Path(r"d:\Computer Science\UoL\Final Project\breast-cancer-detection")
DATA_DIR = ROOT / "data"
DATA_CSV_DIR = DATA_DIR / "csv"
DATA_JPEG_DIR = DATA_DIR / "kaggle/jpeg"

# Checks
print("CSV dir exists:", DATA_CSV_DIR.exists())
print("JPEG dir exists:", DATA_JPEG_DIR.exists())


CSV dir exists: True
JPEG dir exists: True
